In [10]:
import json

with open("all_places.json", "r", encoding="utf-8") as f:
    all_places = json.load(f)

In [4]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import html

def clean_review(text):
    if not isinstance(text, str):
        return None

    text = html.unescape(text)
    text = text.strip()

    if len(text) < 3:
        return None

    if not re.search("[가-힣]", text):  # 한글 없는 외국어 리뷰 제거
        return None

    text = re.sub(r"(.)\1{2,}", r"\1\1", text)  # 반복 문자 정리
    text = re.sub(r"[^\w\s가-힣.,!?]", "", text)  # 이모지, 특수문자 제거
    text = text[:300]  # 너무 긴 리뷰 자르기

    return text if text else None

for place in all_places:
    original_reviews = place.get("reviews", [])
    cleaned_reviews = []

    for review in original_reviews:
        cleaned = clean_review(review)
        if cleaned:
            cleaned_reviews.append(cleaned)

    place["reviews"] = cleaned_reviews  # ✅ 전처리된 리뷰로 덮어쓰기


In [30]:
#user_input_raw = input("여행에서 어떤 걸 원하시나요?\n")
user_input_raw = "공원이 싫어 타워 위주의 여행을 원해. 예 스카이타워"

# 리뷰 전처리 함수 사용
user_cleaned = clean_review(user_input_raw)

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 모델 로드 (이진 분류 기반으로 사전학습된 모델 사용)
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-base")
model.eval()

import re

def is_korean(text):
    return bool(re.search("[가-힣]", text))

def is_valid_review(text):
    if not isinstance(text, str) or len(text.strip()) < 5:
        return False
    else:
        return True

def trimmed_mean(scores, trim_ratio=0.25):
    if len(scores) < 3:
        return sum(scores) / len(scores)
    scores = sorted(scores)
    n = int(len(scores) * trim_ratio)
    trimmed = scores[n:-n] if n > 0 else scores
    return sum(trimmed) / len(trimmed)

# Transformer 기반 적합도 추론 함수
def transformer_score(user_text, place_name, place_reviews, max_review_count=5):
    valid_reviews = [
        r for r in place_reviews
        if is_valid_review(r) and is_korean(r)
    ]
    if not valid_reviews:
        return 0.0

    scores = []
    for review in valid_reviews[:max_review_count]:
        combined_text = f"{place_name}. {review}"  # ⬅️ 명소 이름 포함
        inputs = tokenizer(user_text, combined_text, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            logits = model(**inputs).logits
            score = torch.sigmoid(logits[0][0]).item()
            scores.append(score)

    return round(sum(scores) / len(scores), 4) if scores else 0.0

for place in all_places:
    reviews = place.get("reviews", [])
    place["transformer_score"] = transformer_score(user_cleaned, place["name"], place.get("reviews", []))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
from collections import defaultdict

# 분야별 분류
type_to_places = defaultdict(list)

for place in all_places:
    place_type = place.get("type", "unknown")
    type_to_places[place_type].append(place)

# 분야별 Top 5 출력
print("\n📊 분야별 Transformer 적합도 Top 5:")

for place_type, places in type_to_places.items():
    print(f"\n🔹 {place_type.title()}:")

    sorted_by_score = sorted(places, key=lambda x: x["transformer_score"], reverse=True)
    for i, place in enumerate(sorted_by_score[:], 1):
        print(f"  {i}. {place['name']} ({place['vicinity']})")
        print(f"     적합도: {place['transformer_score']:.4f}, 평점: {place['rating']}, 리뷰 수: {place['user_ratings_total']}")



📊 분야별 Transformer 적합도 Top 5:

🔹 Tourist_Attraction:
  1. 원효대교 (영등포구 여의동)
     적합도: 0.5754, 평점: 4.2, 리뷰 수: 104
  2. 남대문시장 (중구 남대문시장4길 21)
     적합도: 0.5752, 평점: 4.2, 리뷰 수: 27492
  3. 돈의문(서대문) 터 (종로구 신문로2가)
     적합도: 0.5744, 평점: 3.9, 리뷰 수: 51
  4. 덕수궁 (중구 세종대로 99)
     적합도: 0.5742, 평점: 4.6, 리뷰 수: 19738
  5. 목마공원 (양천구 목동 910)
     적합도: 0.5742, 평점: 3.8, 리뷰 수: 108
  6. 서울월드컵경기장 (마포구 월드컵로 240)
     적합도: 0.5738, 평점: 4.4, 리뷰 수: 9215
  7. 가로공원 (양천구 신월동)
     적합도: 0.5735, 평점: 3.5, 리뷰 수: 49
  8. 금천교 (금천구 독산동)
     적합도: 0.5733, 평점: 3.9, 리뷰 수: 169
  9. 궁뜰어린이공원 (서대문구 연희동 123-5)
     적합도: 0.5733, 평점: 4.3, 리뷰 수: 93
  10. 부천레포츠공원 (부천시 원미구 춘의동)
     적합도: 0.5733, 평점: 4.2, 리뷰 수: 48
  11. 매화근린공원 (강서구 등촌동)
     적합도: 0.5732, 평점: 4.1, 리뷰 수: 29
  12. 전쟁기념관 (용산구 이태원로 29)
     적합도: 0.5727, 평점: 4.6, 리뷰 수: 17543
  13. 거리공원 (구로구 공원로 73-1)
     적합도: 0.5727, 평점: 4.2, 리뷰 수: 842
  14. 도당근린공원 백만송이장미원 (부천시 원미구 부천로354번길 100)
     적합도: 0.5726, 평점: 4.2, 리뷰 수: 1501
  15. 방아다리공원 (양천구 신월동)
     적합도: 0.5725, 평점: 4.2, 리뷰 수: 27
 

In [29]:
target_name = "N서울타워"  # 확인하고 싶은 명소 이름

for place in all_places:
    if target_name in place["name"]:
        print(f"📍 {place['name']} ({place['type']})")
        print(f"   적합도: {place.get('transformer_score', '없음')}")

📍 N서울타워 (tourist_attraction)
   적합도: 0.5089


In [ ]:
"""
Transformer를 활용한 사용자 희망 사항과 리뷰 분석 : Fine-Tuning 불가하며 생각보다 훌륭한 성능을 보이지 못하였다고 판단된다.
이에 키워드 기반 점수 생성형태로 점수를 부여하기로 결정

"""